In [60]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, Image, HTML
import cv2
import joblib

df = pd.read_csv('cleaned_data.csv')

df.head()

,Type,Name,Age(months),Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Fee,PetID,PhotoAmt,Breed1_Name,Color1_Name,Color,Breed,AgeGroup,FeeCategory,image_paths
0,Cat,Nibble,3,Male,Small,Short,No,No,No,Healthy,100,86e1089a3,1,Tabby,Black,Black White,Tabby,Kitten,False,['../input/images/86e1089a3-1.jpg']
1,Cat,Unknown,1,Male,Medium,Medium,Not Sure,Not Sure,Not Sure,Healthy,0,6296e909a,2,Domestic Medium Hair,Black,Black Brown,Domestic Medium Hair,Kitten,True,"['../input/images/6296e909a-1.jpg', '../input/..."
2,Dog,Brisco,1,Male,Medium,Medium,Yes,Yes,No,Healthy,0,3422e4906,7,Mixed Breed,Brown,Brown White,Mixed Breed,Puppy,True,"['../input/images/3422e4906-1.jpg', '../input/..."
3,Dog,Miko,4,Female,Medium,Short,Yes,Yes,No,Healthy,150,5842f1ff5,8,Mixed Breed,Black,Black Brown,Mixed Breed,Puppy,False,"['../input/images/5842f1ff5-1.jpg', '../input/..."
4,Dog,Hunter,1,Male,Medium,Short,No,No,No,Healthy,0,850a43f90,3,Mixed Breed,Black,Black,Mixed Breed,Puppy,True,"['../input/images/850a43f90-1.jpg', '../input/..."


In [61]:
data = df.copy()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11875 entries, 0 to 11874
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11875 non-null  object
 1   Name          11875 non-null  object
 2   Age(months)   11875 non-null  int64 
 3   Gender        11875 non-null  object
 4   MaturitySize  11875 non-null  object
 5   FurLength     11875 non-null  object
 6   Vaccinated    11875 non-null  object
 7   Dewormed      11875 non-null  object
 8   Sterilized    11875 non-null  object
 9   Health        11875 non-null  object
 10  Fee           11875 non-null  int64 
 11  PetID         11875 non-null  object
 12  PhotoAmt      11875 non-null  int64 
 13  Breed1_Name   11875 non-null  object
 14  Color1_Name   11875 non-null  object
 15  Color         11875 non-null  object
 16  Breed         11875 non-null  object
 17  AgeGroup      11875 non-null  object
 18  FeeCategory   11875 non-null  bool  
 19  imag

In [62]:
for col in data.select_dtypes('object').columns:
    print(data[col].describe(), '\n')

count     11875
unique        2
top         Dog
freq       6678
Name: Type, dtype: object 

count       11875
unique       5484
top       Unknown
freq         1070
Name: Name, dtype: object 

count      11875
unique         2
top       Female
freq        6652
Name: Gender, dtype: object 

count      11875
unique         4
top       Medium
freq        8119
Name: MaturitySize, dtype: object 

count     11875
unique        3
top       Short
freq       7005
Name: FurLength, dtype: object 

count     11875
unique        3
top         Yes
freq       5366
Name: Vaccinated, dtype: object 

count     11875
unique        3
top         Yes
freq       7339
Name: Dewormed, dtype: object 

count     11875
unique        3
top          No
freq       7556
Name: Sterilized, dtype: object 

count       11875
unique          3
top       Healthy
freq        11389
Name: Health, dtype: object 

count         11875
unique        11875
top       fd80b8c80
freq              1
Name: PetID, dtype: object 

count 

In [63]:
for col in data.select_dtypes('object').columns:
    print(data[col].value_counts(), '\n')

Type
Dog    6678
Cat    5197
Name: count, dtype: int64 

Name
Unknown    1070
Baby         78
Lucky        75
Brownie      75
Mimi         59
           ... 
Rudolph       1
DORA          1
Toothsy       1
Babydog       1
Heidy         1
Name: count, Length: 5484, dtype: int64 

Gender
Female    6652
Male      5223
Name: count, dtype: int64 

MaturitySize
Medium         8119
Small          2616
Large          1112
Extra Large      28
Name: count, dtype: int64 

FurLength
Short     7005
Medium    4126
Long       744
Name: count, dtype: int64 

Vaccinated
Yes         5366
No          4952
Not Sure    1557
Name: count, dtype: int64 

Dewormed
Yes         7339
No          3088
Not Sure    1448
Name: count, dtype: int64 

Sterilized
No          7556
Yes         2888
Not Sure    1431
Name: count, dtype: int64 

Health
Healthy           11389
Minor Injury        459
Serious Injury       27
Name: count, dtype: int64 

PetID
fd80b8c80    1
86e1089a3    1
6296e909a    1
6a73ddcb1    1
e14bf9875 

In [64]:
data.columns

Index(['Type', 'Name', 'Age(months)', 'Gender', 'MaturitySize', 'FurLength',
       'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Fee', 'PetID',
       'PhotoAmt', 'Breed1_Name', 'Color1_Name', 'Color', 'Breed', 'AgeGroup',
       'FeeCategory', 'image_paths'],
      dtype='object')

In [65]:
# Define breed categories
high_demand_breeds = [
    'Bengal', 'Maine Coon', 'Ragdoll', 'Siamese', 'British Shorthair', 'Scottish Fold', 'Sphynx',
    'French Bulldog', 'Golden Retriever', 'Siberian Husky', 'Pomeranian', 'Akita', 'Samoyed'
]
popular_breeds = [
    'Domestic Short Hair', 'Domestic Long Hair', 'Bombay', 'Russian Blue',
    'Labrador Retriever', 'German Shepherd Dog', 'Beagle', 'Boxer', 'Dachshund'
]
common_breeds = [
    'Tabby', 'Calico', 'Tuxedo', 'American Shorthair', 'Mixed Breed', 'Jack Russell Terrier',
    'Border Collie', 'Pit Bull Terrier'
]

# Fee assignment function
def assign_fee(row):
    # Assign base fees based on breed
    if row['Breed1_Name'] in high_demand_breeds:
        fee = np.random.randint(500, 2000)
    elif row['Breed1_Name'] in popular_breeds:
        fee = np.random.randint(200, 800)
    elif row['Breed1_Name'] in common_breeds:
        fee = np.random.randint(50, 300)
    else:
        fee = np.random.randint(50, 300)  # Default for unknown breeds

    # Adjust fee based on age group
    if row['AgeGroup'] == 'Puppy/Kitten':
        fee += np.random.randint(50, 200)  # Higher fee for younger pets
    elif row['AgeGroup'] == 'Adult':
        fee += np.random.randint(0, 50)   # Slight increase for adults
    elif row['AgeGroup'] == 'Senior':
        fee -= np.random.randint(20, 150)  # Discount for seniors

    # Adjust fee based on health
    if row['Health'] == 'Minor Injury':
        fee -= np.random.randint(20, 50)
    elif row['Health'] == 'Serious Injury':
        fee -= np.random.randint(50, 150)

    # Ensure minimum fee is at least $20
    # return max(20, fee)

    return max(20, int(round(fee / 10) * 10))

# Apply the fee assignment function
data['Fee'] = data.apply(assign_fee, axis=1)
data['Fee'].sort_values().unique()

array([  20,   30,   40,   50,   60,   70,   80,   90,  100,  110,  120,
        130,  140,  150,  160,  170,  180,  190,  200,  210,  220,  230,
        240,  250,  260,  270,  280,  290,  300,  310,  320,  330,  340,
        350,  360,  370,  380,  390,  400,  410,  420,  430,  440,  450,
        460,  470,  480,  490,  500,  510,  520,  530,  540,  550,  560,
        570,  580,  590,  600,  610,  620,  630,  640,  650,  660,  670,
        680,  690,  700,  710,  720,  730,  740,  750,  760,  770,  780,
        790,  800,  810,  820,  830,  840,  850,  860,  870,  880,  890,
        900,  910,  920,  930,  940,  950,  960,  970,  980, 1000, 1010,
       1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120,
       1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230,
       1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340,
       1350, 1360, 1380, 1390, 1410, 1420, 1430, 1440, 1450, 1460, 1470,
       1480, 1490, 1500, 1510, 1520, 1530, 1540, 15

In [66]:
# Define fee categories
def categorize_fee(fee):
    if fee <= 200:
        return 'Low'
    elif 200 <= fee <= 600:
        return 'Medium'
    elif 600 <= fee <= 1200:
        return 'High'
    else:
        return 'Premium'

# Apply the fee categorization function
data['FeeCategory'] = data['Fee'].apply(categorize_fee)

# Display the unique values in the new FeeCategory column
data['FeeCategory'].unique()

array(['Low', 'Medium', 'High', 'Premium'], dtype=object)

In [67]:
data['FeeCategory'].value_counts()

FeeCategory
Medium     5316
Low        4919
High       1385
Premium     255
Name: count, dtype: int64

In [68]:
data.isnull().sum()

Type            0
Name            0
Age(months)     0
Gender          0
MaturitySize    0
FurLength       0
Vaccinated      0
Dewormed        0
Sterilized      0
Health          0
Fee             0
PetID           0
PhotoAmt        0
Breed1_Name     0
Color1_Name     0
Color           0
Breed           0
AgeGroup        0
FeeCategory     0
image_paths     0
dtype: int64

In [69]:
# data.to_csv('cleaned_data2.csv', index=False, encoding='utf-8')

In [70]:
ordered_cats = {
    'MaturitySize': ['Not Specified', 'Small', 'Medium', 'Large', 'Extra Large'],
    'FurLength': ['Short', 'Medium', 'Long'],
    'Health': ['Serious Injury', 'Minor Injury', 'Healthy'],
    'FeeCategory': ['Low', 'Medium', 'High', 'Premium']
}

In [71]:
for col in data: 
    if col in ['PhotoAmt']:
        data[col] = data[col].astype('int')
        
    if col in ['Type', 'Gender', 'Health', 'Breed1_Name', 'Breed', 'Color1_Name', 'Color', 'AgeGroup', 'Vaccinated', 'Dewormed', 'Sterilized']:
        data[col] = data[col].astype('category')
        
    if col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        data[col] = data[col].astype(category)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11875 entries, 0 to 11874
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Type          11875 non-null  category
 1   Name          11875 non-null  object  
 2   Age(months)   11875 non-null  int64   
 3   Gender        11875 non-null  category
 4   MaturitySize  11875 non-null  category
 5   FurLength     11875 non-null  category
 6   Vaccinated    11875 non-null  category
 7   Dewormed      11875 non-null  category
 8   Sterilized    11875 non-null  category
 9   Health        11875 non-null  category
 10  Fee           11875 non-null  int64   
 11  PetID         11875 non-null  object  
 12  PhotoAmt      11875 non-null  int64   
 13  Breed1_Name   11875 non-null  category
 14  Color1_Name   11875 non-null  category
 15  Color         11875 non-null  category
 16  Breed         11875 non-null  category
 17  AgeGroup      11875 non-null  category
 18  FeeCat

In [72]:
data.to_csv('cleaned_data2.csv', index=False, encoding='utf-8')

In [73]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

features = ['Type', 'Age(months)', 'Gender', 'MaturitySize', 'FurLength', 'Color1_Name', 'Fee']

# Preprocessing pipeline
numeric_features = ['Age(months)', 'Fee']
categorical_features = ['Type', 'Gender', 'MaturitySize', 'FurLength', 'Color1_Name']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Preprocess the data
X_preprocessed = preprocessor.fit_transform(data[features])

X_preprocessed

array([[-0.47137606, -0.80214191,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.57387914, -0.80214191,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.57387914, -0.80214191,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.19489396, -0.05628578,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.91241552, -0.73110799,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60490628, -0.19835361,  0.        , ...,  0.        ,
         0.        ,  0.        ]], shape=(11875, 20))

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Define user preferences (example)
user_preferences = {
    'Type': 'Cat',
    'Age(months)': 2,
    'Gender': 'Female',
    'MaturitySize': 'Small',
    'FurLength': 'Short',
    # 'FeeCategory': 'Medium',
    'Color1_Name': 'White',
    'Fee': 1100
}

# Convert user preferences to a feature vector
user_input = pd.DataFrame([user_preferences])
user_input_encoded = preprocessor.transform(user_input)

# Compute similarity scores
similarity_scores = cosine_similarity(user_input_encoded, X_preprocessed)

# Rank pets by similarity
data['Similarity'] = similarity_scores[0]
recommended_pets = data.sort_values('Similarity', ascending=False)
recommended_pets = recommended_pets[recommended_pets['Similarity'] > 0.8].head(5)

print(recommended_pets[['Breed1_Name', 'Name', 'Type', 'Gender', 'Age(months)', 'Color', 'Fee']])


               Breed1_Name        Name Type  Gender  Age(months)    Color  Fee
3879               Siamese     Siamese  Cat  Female            2  White    890
10919              Siamese    Snowball  Cat  Female            2  White    790
9308   Domestic Short Hair  Snow White  Cat  Female            3  White    770
7914   Domestic Short Hair        Merc  Cat  Female            1  White    720
1673   Domestic Short Hair        Lulu  Cat  Female            2  White    710


In [75]:
def is_image_clear(image_path, threshold=100):
    """
    Check if an image is clear based on the Laplacian variance method.

    Args:
        image_path (str): Path to the image file.
        threshold (float): Sharpness threshold. Lower values indicate blurry images.

    Returns:
        bool: True if the image is clear, False if it's blurry.
    """
    if not os.path.exists(image_path):
        return False

    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return False

    # Calculate the Laplacian variance
    laplacian_var = cv2.Laplacian(image, cv2.CV_64F).var()
    return laplacian_var > threshold

In [76]:
# # Displaying recommended pets
# print("Recommended Pets (Images)")
# for _, pet in recommended_pets.iterrows():
#     print(f"**{pet['Name']}** - {pet['Breed1_Name']} (Similarity: {pet['Similarity']:.2f})")

#     # Check if 'image_paths' exists and is a valid string
#     if isinstance(pet['image_paths'], str):
#         image_paths = pet['image_paths'].strip("[]").replace("'", "").split(", ")
#         image_paths = image_paths[:3]  # Limit to 3 images per pet for display
        
#         # Display images if paths exist
#         for image_path in image_paths:
#             image_path = os.path.abspath(image_path)
#             if os.path.exists(image_path):
#                 display(Image(image_path, width=200))  # Display image
#             else:
#                 print(f"Image not found: {image_path}")

#     print("-" * 140)  # Separator for each pet

In [77]:
data.head()

,Type,Name,Age(months),Gender,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,...,PetID,PhotoAmt,Breed1_Name,Color1_Name,Color,Breed,AgeGroup,FeeCategory,image_paths,Similarity
0,Cat,Nibble,3,Male,Small,Short,No,No,No,Healthy,...,86e1089a3,1,Tabby,Black,Black White,Tabby,Kitten,Low,['../input/images/86e1089a3-1.jpg'],0.115797
1,Cat,Unknown,1,Male,Medium,Medium,Not Sure,Not Sure,Not Sure,Healthy,...,6296e909a,2,Domestic Medium Hair,Black,Black Brown,Domestic Medium Hair,Kitten,Low,"['../input/images/6296e909a-1.jpg', '../input/...",-0.105885
2,Dog,Brisco,1,Male,Medium,Medium,Yes,Yes,No,Healthy,...,3422e4906,7,Mixed Breed,Brown,Brown White,Mixed Breed,Puppy,Low,"['../input/images/3422e4906-1.jpg', '../input/...",-0.219241
3,Dog,Miko,4,Female,Medium,Short,Yes,Yes,No,Healthy,...,5842f1ff5,8,Mixed Breed,Black,Black Brown,Mixed Breed,Puppy,Medium,"['../input/images/5842f1ff5-1.jpg', '../input/...",0.177665
4,Dog,Hunter,1,Male,Medium,Short,No,No,No,Healthy,...,850a43f90,3,Mixed Breed,Black,Black,Mixed Breed,Puppy,Low,"['../input/images/850a43f90-1.jpg', '../input/...",-0.016716


In [78]:
df = pd.read_csv("cleaned_data2.csv")  

df.columns

Index(['Type', 'Name', 'Age(months)', 'Gender', 'MaturitySize', 'FurLength',
       'Vaccinated', 'Dewormed', 'Sterilized', 'Health', 'Fee', 'PetID',
       'PhotoAmt', 'Breed1_Name', 'Color1_Name', 'Color', 'Breed', 'AgeGroup',
       'FeeCategory', 'image_paths'],
      dtype='object')

# Price Prediction

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Define features and target
features2 = [
    'Type', 'Gender', 'AgeGroup', 'MaturitySize', 'FurLength',
    'Color1_Name', 'Breed1_Name', 'FeeCategory'
]
target = 'Fee'  # Target column for price prediction

X = data[features2]
y = data[target]

# One-hot encode categorical features (using OneHotEncoder)
encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
X = encoder.fit_transform(X)

# Save the encoder
joblib.dump(encoder, 'encoder.pkl')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=300, max_depth=20, min_samples_leaf=2, min_samples_split=2, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest Regression Results:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")


Random Forest Regression Results:
Mean Squared Error: 6082.73
Root Mean Squared Error: 77.99
R^2 Score: 0.92


In [84]:
# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Optimized Random Forest Regression Results:")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")


Optimized Random Forest Regression Results:
Mean Squared Error: 6082.73
Root Mean Squared Error: 77.99
R^2 Score: 0.92


In [87]:
joblib.dump(rf_model, 'random_forest_model.pkl')

['random_forest_model.pkl']